In [85]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [86]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [87]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: 无法创建目录"/home/aistudio/external-libraries": 文件已存在
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/9c/d8/909c4089dbe4ade9f9705f143c9f13f065049a9d5e7d34c828aefdd0a97c/beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/16/e3/4ad79882b92617e3a4a0df1960d6bce08edfb637737ac5c3f3ba29022e25/soupsieve-2.3.2.post1-py3-none-any.whl (37 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [88]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
import io
import os
import sys
import requests
from collections import OrderedDict
import math
import numpy as np
import paddle 
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import Embedding



sys.path.append('/home/aistudio/external-libraries')

In [89]:
def download():
    corpus_url ="https://dataset.bj.bcebos.com/word2vec/text8.txt"
    web_request=requests.get(corpus_url)
    corpus=web_request.content

    with open("./text8.txt","wb") as f:
        f.write(corpus)
    f.close()

download()    

In [90]:
#读取text8数据
def load_text8():
    with open("./text8.txt","r") as f:
        corpus =f.read().strip('/n')
    f.close()
    return corpus

corpus = load_text8()


print(corpus[:500])

 

 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philoso


In [91]:
def data_preprocess(corpus):

    corpus=corpus.strip().lower()
    corpus=corpus.split(" ")#切分 空格

    return corpus

corpus = data_preprocess(corpus)
print(corpus[:50])


['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


In [92]:
def build_dict(corpus):
    #首先统计每个不同词的频率
    word_fre_dict=dict()
    for word in corpus:
        if word not in word_fre_dict:
            word_fre_dict[word]=0
        word_fre_dict[word]+=1
            #高频词排序靠前 一般来说高频词是 i等代词，频率低的次往往是一些名词，如，nlp
            
    word_fre_dict=sorted(word_fre_dict.items(),key= lambda x:x[1],reverse=True)
    
    print(word_fre_dict)
#词到id的映射关系：word2id_dict
#每个id出现的频率：word2id_freq
#每个id到词典映射关系:id2word_dict
    word2id_dict=dict()
    word2id_freq=dict()
    id2word_dict=dict() 
#按照频率，从高到低开始遍历每个单词，提供唯一id
    for word,freq in word_fre_dict:
        curr_id=len(word2id_dict)
        word2id_dict[word]=curr_id
        word2id_freq[word2id_dict[word]]=freq
        id2word_dict[curr_id]=word

    return word2id_freq,word2id_dict,id2word_dict



word2id_freq,word2id_dict,id2word_dict=build_dict(corpus)
vocab_size=len(word2id_freq)
print("there are totally %d different words in the corpus" % vocab_size)

for _,(word,word_id) in zip(range(50),word2id_dict.items()):
    print("word %s,its id %d,its word freq %d" % (word,word_id,word2id_freq[word_id]))            

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



there are totally 253854 different words in the corpus
word the,its id 0,its word freq 1061396
word of,its id 1,its word freq 593677
word and,its id 2,its word freq 416629
word one,its id 3,its word freq 411764
word in,its id 4,its word freq 372201
word a,its id 5,its word freq 325873
word to,its id 6,its word freq 316376
word zero,its id 7,its word freq 264975
word nine,its id 8,its word freq 250430
word two,its id 9,its word freq 192644
word is,its id 10,its word freq 183153
word as,its id 11,its word freq 131815
word eight,its id 12,its word freq 125285
word for,its id 13,its word freq 118445
word s,its id 14,its word freq 116710
word five,its id 15,its word freq 115789
word three,its id 16,its word freq 114775
word was,its id 17,its word freq 112807
word by,its id 18,its word freq 111831
word that,its id 19,its word freq 109510
word four,its id 20,its word freq 108182
word six,its id 21,its word freq 102145
word seven,its id 22,its word freq 99683
word with,its id 23,its word freq 

In [93]:
#语料转换成id序列

def convert_corpus_to_id(corpus,word2id_dict):
    corpus=[word2id_dict[word] for word in corpus]
    return corpus

corpus = convert_corpus_to_id(corpus,word2id_dict)

print("%d tokens in the corpus" % len(corpus))
print(corpus[:50])

17005207 tokens in the corpus
[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580, 0, 194, 10, 190, 58, 4, 5, 10712, 214, 6, 1324, 104, 454, 19, 58, 2731, 362, 6, 3672, 0]


In [94]:
#二次采样算法，优化词分布 强化训练效果

def subsampling(corpus,word2id_freq):
    #这个discard函数决定了一个词会不会被替换，这个函数具体有随机性，每次提调用结构不同
    #如果一个词的频率很大，那么它被遗弃的概率很大
    def discard(word_id):
        return random.uniform(0,1)< 1-math.sqrt(le-4/word2id_freq * len(corpus))

    corpus = subsampling(corpus,word2id_freq)
    return corpus



In [108]:
#max_window_size代表了最大word_size打大小，程序会根据max_window_size从左到右扫描整个语料
#negative_sample_num代表了对于每个正样本，我么需要随机采样多少负样本用于训练，
#一般来说，negative_sample_num越大，训练效果越稳定，但训练速度越慢

import random
def build_data(corpus,word2id_dict,word2id_freq,max_window_size=3,negative_sample_num=4):
    #使用一个list存储处理好的的数据
    dataset=[]

    #从左到右开始枚举中心点的位置
    for center_word_idx in range(len(corpus)):
        #以max_window_size为最大值,随机采样一个window_size
        window_size=random.randint(1,max_window_size)
        #当前中心词为中心，左右两侧在window_size内的词都可以看成是正样本
        center_word=corpus[center_word_idx]
        
        #以当前中心词为中心，左右两侧在window_size内都可以看做是正样本
        postive_word_range=(max(0,center_word_idx-window_size),min(len(corpus)-1,center_word_idx+ window_size))
        postive_word_candidates=[corpus[idx] for idx in range(postive_word_range[0],postive_word_range[1]+1) if idx!=center_word_idx]


        #对于每个正样本随机采样n_s_n个负样本
        
        for postive_word in postive_word_candidates:
            #先把（中心词，正样本，label==1）三元数组放入dataset中
            #这里label=1表示这个样本是正样本
            dataset.append((center_word,postive_word,1))

            #负采样
            i=0
            while i<negative_sample_num:
                negative_word_candidate=random.randint(0,vocab_size-1)

                if negative_word_candidate not in postive_word_candidates:
                    #把（中心词，正样本,label=0）的三元数组放入dataset中,
                    #这里lable=0代表负样本

                    dataset.append((center_word,negative_word_candidate,0))
                    i+=1

    return dataset

In [109]:
#构造mini_batch 准备对模型进行训练
#我们将不同类型的数据放到不同的tensor礼，便于神经网络进行处理
#并通过 numpy的array函数 构造不同的tensor 把这些tensor送入神经网络当周


def build_batch(dataset,batch_size,epoch_num):
    #center_word_batch缓存batch_size个中心词
    center_word_batch=[]
    #target_word_batch缓存batch_size个目标词(正样本或者负样本)
    target_word_batch=[]
    #label_batch缓存batch_size个0或1标签用于模型训练
    label_batch=[]

    for epoch in range(epoch_num):
        #每个新epoch_num之前，都对数据进行一次随机打乱，提高训练效果
        random.shuffle(dataset)

        for center_word,target_word,label in dataset:
            #遍历dataset中的每个样本，并将这些数据送到不同的tensor中

            center_word_batch.append([center_word])
            target_word_batch.append([target_word])
            label_batch.append(label)

            #当样本积攒到一个batch_size后，把数据都返回
            #使用np的array函数把list封装成tensor
            #并使用python的迭代器机制，把数据yield
            #使用迭代器的好处是节省内存

            if len(center_word_batch) == batch_size:
                yield np.array(center_word_batch).astype("int64"),\
                    np.array(target_word_batch).astype("int64"),\
                    np.array(label_batch).astype("float32")
                center_word_batch=[]
                target_word_batch=[]
                label_batch=[]
            
        if len(center_word_batch) >0:
            yield np.array(center_word_batch).astype("int64"),\
                np.array(target_word_batch).astype("int64"),\
                np.array(label_batch).astype("float32")



In [110]:
#定义skip-gram网络
class SkipGram(fluid.dygraph.Layer):
    def __init__(self,vocab_size,embedding_size,init_scale=0.1):

        super(SkipGram,self).__init__()
        self.vocab_size=vocab_size
        self.embedding_size=embedding_size
#构造词向量参数
        self.embedding=Embedding(
            size=[self.vocab_size,self.embedding_size],
            dtype="float32",
            param_attr=fluid.ParamAttr(
                name="embedding_para",
                initializer=fluid.initializer.UniformInitializer(
                    low=-0.5/embedding_size,high=0.5/embedding_size
                )
            )
        )


        self.embedding_out=Embedding(
            size=[self.vocab_size,self.embedding_size],
            dtype='float32',
            param_attr=fluid.ParamAttr(
                name="embedding_out_para",
                initializer=fluid.initializer.UniformInitializer(
                    low=-0.5/embedding_size,
                )
            )
        )

In [111]:

#前馈
def forward(self,center_words,target_words,label):
    center_words_emb=self.embedding(center_words)
    target_words_emb=self.embedding_out(target_words)
    #点乘运算
    word_sim= fluid.layers.elementwise_mul(center_words_emb,target_words_emb)
    word_sim=fluid.layers.reduce_sum(word_sim,dim=-1)
    word_sim=fluid.layers.reshape(word_sim,shape=[-1])
    pred=fluid.layers.sigmoid(word_sim)

    #sigmoid和cross entropy合成以后还可以尽可能的优化。
    loss=fluid.layers.sigmod_crosss_entropy_with_logits(word_sim,label)
    loss=fluid.layers.reduce_mean(loss)

    return pred,loss


In [112]:
batch_size=512
epoch_num=3
embedding_size=200
step=0
learning_rate=0.001

#embed 学习到的embedding参数
def get_similar_tokens(query_token,k,embed):
    W=embed.numpy()
    X=W[word2id_dict[query_token]]
    cos=np.dot(W,X)/nq.sqrt(np.sum(W*W,axis=1)* np.sum(x*x)+1e-9)
    flat=cos.flatten()
    indices=np.argpartition(flat,-k)[-k:]
    indices=indices[np.argsort(-flat[indices])]
    for i in indices:
        print('for word %s,the similar word is %s' % (query_token,str(id2word_dict[i])))


In [ ]:
with fluid.dygraph.guard(fluid.CPUPlace()):
    dataset=build_data(corpus,word2id_dict,word2id_freq,max_window_size=3,negative_sample_num=4)
    
    skip_gram_model=SkipGram(vocab_size,embedding_size)

    adam=fluid.optimizer.AdamOptimizer(learning_rate=learning_rate,parameter_list=skip_gram_model.parameters())

    for center_words,target_words,label in build_batch(dataset,batch_size,epoch_num):
        center_words_vars=fluid.dygraph.to_variable(center_words)
        target_words_vars=fluid.dygraph.to_variable(target_words)
        label_var=fluid.dygraph.to_variable(label)

        pred,loss-skip_gram_model(center_words_vars,target_words_vars,label_var)

        loss.backward()

        adam.minimize(loss)

        skip_gram_model.clear_gradients()

        step+=1
        if step %100==0:
            print("step %d,loss %.3f" % (step,loss.numpy()[0]))

        if step %10000 ==0:
            get_similar_tokens("one",5,skip_gram_model.embedding.weight)
            get_similar_tokens('she',5,skip_gram_model.embedding.weight)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 